In [7]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
import numpy as np
import plotly.express as px
import phate
import scprep
import pubchempy as pcp
import umap
import seaborn as sns
from sklearn.decomposition import PCA
import time
import json
import sqlite3
import os
%matplotlib inline

# Data Processing

Import files:
<ul>
<li>./GDSC2_drugsens/GDSC2_fitted_dose_response_27Oct23.xlsx </li>
</ul>

Export files:
<ul>
<li>GeneExp_CCLE_prep.csv</li>
<li>DrugSens_GDSC_prep.csv</li>

In [8]:
GDSC2 = pd.read_excel("./GDSC2_drugsens/GDSC2_fitted_dose_response_27Oct23.xlsx")
CCLE = pd.read_csv("./CCLE-DepMap22Q2_geneexp/sample_info.csv")

In [9]:
GDSC2 

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,343,15946310,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.463887,0.930220,0.089052,0.433123
1,GDSC2,343,15946548,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869455,0.614970,0.111351,-1.421100
2,GDSC2,343,15946830,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360586,0.791072,0.142855,-0.599569
3,GDSC2,343,15947087,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.044940,0.592660,0.135539,-1.516647
4,GDSC2,343,15947369,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741991,0.734047,0.128059,-0.807232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242031,GDSC2,343,16188242,1659928,SNU-175,SIDM00216,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.127082,0.976746,0.074498,0.156872
242032,GDSC2,343,16188695,1660034,SNU-407,SIDM00214,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,8.576377,0.913378,0.057821,-1.626959
242033,GDSC2,343,16188953,1660035,SNU-61,SIDM00194,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.519636,0.975001,0.058090,0.608442
242034,GDSC2,343,16189493,1674021,SNU-C5,SIDM00498,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.694579,0.969969,0.101013,0.809684


In [10]:
CCLE 

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
0,ACH-000016,SLR 21,SLR21,SLR21_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_V607,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-JnARLB,NaN,Clear cell renal cell carcinoma,C4033,NaN
1,ACH-000032,MHH-CALL-3,MHHCALL3,MHHCALL3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,DSMZ,CVCL_0089,NaN,...,b_cell,NaN,NaN,NaN,NaN,PT-p2KOyI,NaN,Childhood B acute lymphoblastic leukemia,C9140,NaN
2,ACH-000033,NCI-H1819,NCIH1819,NCIH1819_LUNG,NaN,NaN,Female,Academic lab,CVCL_1497,NaN,...,NSCLC_adenocarcinoma,NaN,NaN,NaN,NaN,PT-9p1WQv,NaN,Lung adenocarcinoma,C3512,NaN
3,ACH-000043,Hs 895.T,HS895T,HS895T_FIBROBLAST,NaN,NaN,Female,ATCC,CVCL_0993,NaN,...,NaN,NaN,2D: adherent,NaN,NaN,PT-rTUVZQ,NaN,Melanoma,C3224,NaN
4,ACH-000049,HEK TE,HEKTE,HEKTE_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_WS59,NaN,...,NaN,NaN,NaN,immortalized,NaN,PT-qWYYgr,NaN,NaN,NaN,No information is available about this cell li...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,ACH-002393,CRO-AP3,CROAP3,CROAP3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,Sanger,CVCL_1810,NaN,...,b_cell_primary_effusion,NaN,NaN,NaN,NaN,PT-TC0lZM,NaN,Primary effusion lymphoma,C6915,NaN
1836,ACH-002394,GEO,GEO,GEO_LARGE_INTESTINE,NaN,NaN,NaN,Sanger,CVCL_0271,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-Fa1q9q,NaN,Colon carcinoma,C4910,NaN
1837,ACH-002395,HuH-6 Clone 5,HUH6CLONE5,HUH6CLONE5_LIVER,NaN,NaN,Male,Sanger,CVCL_1296,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-TtIXsL,ACH-000671,Hepatoblastoma,C3728,NaN
1838,ACH-002396,Sarc9371,SARC9371,SARC9371_BONE,NaN,NaN,NaN,Sanger,CVCL_5G89,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-715FdC,NaN,Osteosarcoma,C9145,NaN


In [11]:
GDSC2_filt_col = GDSC2[["DATASET", "COSMIC_ID", "DRUG_ID", "DRUG_NAME", "LN_IC50"]]

In [12]:
print(GDSC2_filt_col["DRUG_NAME"].nunique())
print(GDSC2_filt_col["DRUG_ID"].nunique())
# one drug name != one drug id 

286
295


In [13]:
GDSC2["DRUG_NAME"] = GDSC2["DRUG_NAME"].str.upper()

In [14]:
processed_drug_df = GDSC2.loc[~GDSC2.duplicated(subset=['DRUG_NAME','DRUG_ID'],keep='first'),['DRUG_ID','DRUG_NAME']]
print(processed_drug_df['DRUG_ID'].nunique())
print(processed_drug_df['DRUG_NAME'].nunique())

295
286


In [15]:
GDSC_sorted = GDSC2.sort_values(by = ["DRUG_NAME", "COSMIC_ID", "DATASET"], ascending=True)

In [16]:
GDSC_drop_dup = GDSC_sorted.drop_duplicates(subset=["DRUG_NAME", "COSMIC_ID"])
GDSC_drop_dup["DRUG_NAME"].nunique()
GDSC_drop_dup["COSMIC_ID"].nunique()
GDSC_drop_dup.isna().sum()

DATASET                0
NLME_RESULT_ID         0
NLME_CURVE_ID          0
COSMIC_ID              0
CELL_LINE_NAME         0
SANGER_MODEL_ID        0
TCGA_DESC           1066
DRUG_ID                0
DRUG_NAME              0
PUTATIVE_TARGET    26438
PATHWAY_NAME           0
COMPANY_ID             0
WEBRELEASE             0
MIN_CONC               0
MAX_CONC               0
LN_IC50                0
AUC                    0
RMSE                   0
Z_SCORE                0
dtype: int64

In [17]:
CCLE_dropna = CCLE[["DepMap_ID", "COSMICID"]].dropna()
CCLE_dropna["COSMICID"] = CCLE_dropna["COSMICID"].astype(int) 
CCLE_map = CCLE_dropna.copy()
dict_CCLE = dict(zip(CCLE_map["COSMICID"], CCLE_map["DepMap_ID"]))

In [18]:
GDSC_drop_dup["COSMIC_ID"] = GDSC_drop_dup["COSMIC_ID"].map(dict_CCLE)

/var/folders/dx/n9ptww816r99pv93j_qb32hr0000gn/T/ipykernel_25815/3808298915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GDSC_drop_dup["COSMIC_ID"] = GDSC_drop_dup["COSMIC_ID"].map(dict_CCLE)


In [19]:
GDSC_map= GDSC_drop_dup.dropna(subset = ["COSMIC_ID"])
GDSC_map.isna().sum()
GDSC_map

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
155759,GDSC2,343,15946463,ACH-001711,PFSK-1,SIDM01132,MB,1824,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.100251,0.972302,0.025546,-0.955691
155760,GDSC2,343,15946728,ACH-000052,A673,SIDM00848,UNCLASSIFIED,1824,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.705026,0.984602,0.087033,-0.224533
155761,GDSC2,343,15947015,ACH-002106,ES5,SIDM00263,UNCLASSIFIED,1824,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.609253,0.983136,0.025179,-0.340319
155762,GDSC2,343,15947267,ACH-002108,ES7,SIDM00269,UNCLASSIFIED,1824,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.227487,0.977563,0.031422,-0.801866
155763,GDSC2,343,15947549,ACH-002111,EW-11,SIDM00203,UNCLASSIFIED,1824,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.257776,0.963789,0.075202,-0.765246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144330,GDSC2,343,16188583,ACH-000955,SNU-407,SIDM00214,COREAD,1802,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,3.834554,0.976913,0.058652,-0.030708
144331,GDSC2,343,16188853,ACH-000532,SNU-61,SIDM00194,COREAD,1802,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,4.399959,0.985227,0.028882,0.584560
144332,GDSC2,343,16189121,ACH-000991,SNU-81,SIDM00193,COREAD,1802,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,3.612773,0.990295,0.059795,-0.272048
144333,GDSC2,343,16189372,ACH-000970,SNU-C5,SIDM00498,COREAD,1802,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,4.742368,0.983071,0.046014,0.957166


In [20]:
GDSC_mixing = GDSC_map.rename(columns={"COSMIC_ID" : "cell_line"})
GDSC_temp = GDSC_mixing.drop(columns=["DRUG_ID", "DATASET"])
GDSC_temp = GDSC_temp.reset_index(drop=True)
GDSC_temp

,NLME_RESULT_ID,NLME_CURVE_ID,cell_line,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,343,15946463,ACH-001711,PFSK-1,SIDM01132,MB,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.100251,0.972302,0.025546,-0.955691
1,343,15946728,ACH-000052,A673,SIDM00848,UNCLASSIFIED,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.705026,0.984602,0.087033,-0.224533
2,343,15947015,ACH-002106,ES5,SIDM00263,UNCLASSIFIED,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.609253,0.983136,0.025179,-0.340319
3,343,15947267,ACH-002108,ES7,SIDM00269,UNCLASSIFIED,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.227487,0.977563,0.031422,-0.801866
4,343,15947549,ACH-002111,EW-11,SIDM00203,UNCLASSIFIED,123138,NaN,Unclassified,1043,Y,0.001001,1.0,2.257776,0.963789,0.075202,-0.765246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230608,343,16188583,ACH-000955,SNU-407,SIDM00214,COREAD,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,3.834554,0.976913,0.058652,-0.030708
230609,343,16188853,ACH-000532,SNU-61,SIDM00194,COREAD,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,4.399959,0.985227,0.028882,0.584560
230610,343,16189121,ACH-000991,SNU-81,SIDM00193,COREAD,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,3.612773,0.990295,0.059795,-0.272048
230611,343,16189372,ACH-000970,SNU-C5,SIDM00498,COREAD,ZOLEDRONATE,,Other,1043,Y,0.002001,2.0,4.742368,0.983071,0.046014,0.957166


In [21]:
drug_name_forsmile = GDSC_temp["DRUG_NAME"].unique().tolist()

## Import SMILES from PubChemPy

In [22]:
! pip install tqdm

In [23]:
from tqdm import tqdm
import pubchempy as pcp

def get_smiles_from_names(compound_names):
    """
    Given a list of compound names, retrieve their SMILES strings using PubChemPy.
    Returns a dictionary where keys are compound names and values are SMILES strings.
    """
    smiles_dict = {}
    
    for name in tqdm(compound_names, desc="Retrieving SMILES", unit="compound"):
        # Use the 'name' namespace to search by compound name
        results = pcp.get_compounds(name, 'name')
        
        if results:
            # Take the first matching compound
            compound = results[0]
            # Check if the compound has a canonical SMILES
            if compound.canonical_smiles:
                smiles_dict[name] = compound.canonical_smiles
            else:
                smiles_dict[name] = "SMILES not found."
        else:
            smiles_dict[name] = "No compound found."
    
    return smiles_dict


In [24]:
drug_name_forsmile

['123138',
 '123829',
 '150412',
 '5-AZACYTIDINE',
 '5-FLUOROURACIL',
 '50869',
 '615590',
 '630600',
 '667880',
 '720427',
 '729189',
 '741909',
 '743380',
 '765771',
 '776928',
 'A-366',
 'ABT737',
 'ACETALAX',
 'AFATINIB',
 'AFURESERTIB',
 'AGI-5198',
 'AGI-6780',
 'AGK2',
 'ALISERTIB',
 'ALPELISIB',
 'ALPHA-LIPOIC ACID',
 'AMG-319',
 'ASCORBATE (VITAMIN C)',
 'AT13148',
 'AVAGACESTAT',
 'AXITINIB',
 'AZ6102',
 'AZ960',
 'AZD1208',
 'AZD1332',
 'AZD2014',
 'AZD3759',
 'AZD4547',
 'AZD5153',
 'AZD5363',
 'AZD5438',
 'AZD5582',
 'AZD5991',
 'AZD6482',
 'AZD6738',
 'AZD7762',
 'AZD8055',
 'AZD8186',
 'BDF00022089A',
 'BDILV000379A',
 'BDOCA000347A',
 'BDP-00009066',
 'BEN',
 'BI-2536',
 'BIBR-1532',
 'BICALUTAMIDE',
 'BLEOMYCIN',
 'BLEOMYCIN (50 UM)',
 'BMS-345541',
 'BMS-536924',
 'BMS-754807',
 'BORTEZOMIB',
 'BOSUTINIB',
 'BPD-00008900',
 'BROMOSPORINE',
 'BUPARLISIB',
 'BX795',
 'CAMPTOTHECIN',
 'CCT-018159',
 'CCT007093',
 'CDK9_5038',
 'CDK9_5576',
 'CEDIRANIB',
 'CHIR-99021',
 '

In [25]:
# Retrieve SMILES for each name
smiles_results = get_smiles_from_names(drug_name_forsmile)

Retrieving SMILES: 100%|██████████| 286/286 [11:30<00:00,  2.41s/compound]


In [26]:
smiles_results

{'123138': 'No compound found.',
 '123829': 'No compound found.',
 '150412': 'No compound found.',
 '5-AZACYTIDINE': 'C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N',
 '5-FLUOROURACIL': 'C1=C(C(=O)NC(=O)N1)F',
 '50869': 'No compound found.',
 '615590': 'No compound found.',
 '630600': 'No compound found.',
 '667880': 'No compound found.',
 '720427': 'No compound found.',
 '729189': 'No compound found.',
 '741909': 'No compound found.',
 '743380': 'No compound found.',
 '765771': 'No compound found.',
 '776928': 'No compound found.',
 'A-366': 'COC1=C(C=C2C(=C1)C3(CCC3)C(=N2)N)OCCCN4CCCC4',
 'ABT737': 'CN(C)CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)NC(=O)C3=CC=C(C=C3)N4CCN(CC4)CC5=CC=CC=C5C6=CC=C(C=C6)Cl)[N+](=O)[O-]',
 'ACETALAX': 'CC(=O)OC1=CC=C(C=C1)C2(C3=CC=CC=C3NC2=O)C4=CC=C(C=C4)OC(=O)C',
 'AFATINIB': 'CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)OC4CCOC4',
 'AFURESERTIB': 'CN1C(=C(C=N1)Cl)C2=C(SC(=C2)C(=O)NC(CC3=CC(=CC=C3)F)CN)Cl',
 'AGI-5198': 'CC1=CC=CC=C1C(C(=O)NC2CCCCC2)N(C

In [27]:
# Print the results
for name, smiles in smiles_results.items():
    print(f"{name}: {smiles}")

123138: No compound found.
123829: No compound found.
150412: No compound found.
5-AZACYTIDINE: C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
5-FLUOROURACIL: C1=C(C(=O)NC(=O)N1)F
50869: No compound found.
615590: No compound found.
630600: No compound found.
667880: No compound found.
720427: No compound found.
729189: No compound found.
741909: No compound found.
743380: No compound found.
765771: No compound found.
776928: No compound found.
A-366: COC1=C(C=C2C(=C1)C3(CCC3)C(=N2)N)OCCCN4CCCC4
ABT737: CN(C)CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)NC(=O)C3=CC=C(C=C3)N4CCN(CC4)CC5=CC=CC=C5C6=CC=C(C=C6)Cl)[N+](=O)[O-]
ACETALAX: CC(=O)OC1=CC=C(C=C1)C2(C3=CC=CC=C3NC2=O)C4=CC=C(C=C4)OC(=O)C
AFATINIB: CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)OC4CCOC4
AFURESERTIB: CN1C(=C(C=N1)Cl)C2=C(SC(=C2)C(=O)NC(CC3=CC(=CC=C3)F)CN)Cl
AGI-5198: CC1=CC=CC=C1C(C(=O)NC2CCCCC2)N(C3=CC(=CC=C3)F)C(=O)CN4C=CN=C4C
AGI-6780: C1CC1NS(=O)(=O)C2=CC(=C(C=C2)C3=CSC=C3)NC(=O)NC4=CC=CC(=C4)C(F)(F)F
AGK2: C1=CC2=C(

In [28]:
GDSC_temp = GDSC_temp.drop(columns=["NLME_RESULT_ID", "NLME_CURVE_ID", "SANGER_MODEL_ID", "TCGA_DESC", "PUTATIVE_TARGET",
                                    "PATHWAY_NAME", "COMPANY_ID", "WEBRELEASE", "MIN_CONC", "MAX_CONC", "AUC", "RMSE", 
                                    "Z_SCORE"])
GDSC_temp = GDSC_temp.reset_index(drop=True)
GDSC_temp

,cell_line,CELL_LINE_NAME,DRUG_NAME,LN_IC50
0,ACH-001711,PFSK-1,123138,2.100251
1,ACH-000052,A673,123138,2.705026
2,ACH-002106,ES5,123138,2.609253
3,ACH-002108,ES7,123138,2.227487
4,ACH-002111,EW-11,123138,2.257776
...,...,...,...,...
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773
230611,ACH-000970,SNU-C5,ZOLEDRONATE,4.742368


In [29]:
GDSC_temp["SMILES"] = GDSC_temp["DRUG_NAME"].map(smiles_results)
GDSC_getsmile = GDSC_temp.dropna()
GDSC_getsmile = GDSC_getsmile[~GDSC_getsmile['SMILES'].isin(["No compound found.", "SMILES not found."])]
GDSC_getsmile

,cell_line,CELL_LINE_NAME,DRUG_NAME,LN_IC50,SMILES
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
2104,ACH-002108,ES7,5-AZACYTIDINE,4.432358,C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
2105,ACH-002111,EW-11,5-AZACYTIDINE,4.013135,C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,C1=NC(=NC(=O)N1C2C(C(C(O2)CO)O)O)N
...,...,...,...,...,...
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O
230611,ACH-000970,SNU-C5,ZOLEDRONATE,4.742368,C1=CN(C=N1)CC(O)(P(=O)(O)O)P(=O)(O)O


In [30]:
processed_drug_df['SMILES'] = processed_drug_df['DRUG_NAME'].map(smiles_results)
processed_drug_df = processed_drug_df.dropna()
processed_drug_df = processed_drug_df[~processed_drug_df['SMILES'].isin(["No compound found.", "SMILES not found."])]
processed_drug_df

,DRUG_ID,DRUG_NAME,SMILES
0,1003,CAMPTOTHECIN,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
968,1004,VINBLASTINE,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...
1709,1005,CISPLATIN,N.N.Cl[Pt]Cl
2469,1006,CYTARABINE,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O
3212,1007,DOCETAXEL,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...
...,...,...,...
235232,2177,SGC0946,CC(C)N(CCCNC(=O)NC1=CC=C(C=C1)C(C)(C)C)CC2C(C(...
236178,2359,GSK2830371,CC1=C(C=C(C=N1)Cl)NCC2=CC=C(S2)C(=O)NC(CC3CCCC...
239834,2439,GLUTATHIONE,C(CC(=O)NC(CS)C(=O)NCC(=O)O)C(C(=O)O)N
240566,2498,ALPHA-LIPOIC ACID,C1CSSC1CCCCC(=O)O


In [31]:
unique_smiles = GDSC_getsmile["SMILES"].unique().tolist()
remover = SaltRemover()
norm_smiles_dict = {smiles: Chem.MolToSmiles(remover.StripMol(Chem.MolFromSmiles(smiles), dontRemoveEverything=True)) for smiles in unique_smiles}
GDSC_cleaned = GDSC_getsmile.copy()
GDSC_cleaned["SMILES"]=GDSC_cleaned["SMILES"].map(norm_smiles_dict) 

In [32]:
GDSC_cleaned

,cell_line,CELL_LINE_NAME,DRUG_NAME,LN_IC50,SMILES
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2104,ACH-002108,ES7,5-AZACYTIDINE,4.432358,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2105,ACH-002111,EW-11,5-AZACYTIDINE,4.013135,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
...,...,...,...,...,...
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230611,ACH-000970,SNU-C5,ZOLEDRONATE,4.742368,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O


## Omics Selection

In [33]:
GeneExp = pd.read_csv("./CCLE-DepMap22Q2_geneexp/CCLE_expression_cleaned.csv", index_col = 0)

In [34]:
GeneExp_CellLine = set(GeneExp.index)

DrugSens_CellLine = set(GDSC_cleaned['cell_line'])
MultiOmics_CellLine = GeneExp_CellLine.intersection(DrugSens_CellLine)

In [35]:
GDSC_selectomics = GDSC_cleaned.loc[GDSC_cleaned['cell_line'].isin(MultiOmics_CellLine)]

# rename columns
GDSC_selectomics = GDSC_selectomics.rename(columns={"DRUG_NAME":"gdsc_name", "LN_IC50":"IC50"})
GDSC_selectomics

,cell_line,CELL_LINE_NAME,gdsc_name,IC50,SMILES
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2108,ACH-000905,5637,5-AZACYTIDINE,3.783443,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
2109,ACH-000242,RT4,5-AZACYTIDINE,2.648009,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1
...,...,...,...,...,...
230607,ACH-000989,SNU-175,ZOLEDRONATE,3.225748,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O


In [36]:
GDSC_selectomics['gdsc_name'].nunique()

228

## Map Cancer type

In [37]:
cell_line_info = pd.read_csv('./CCLE-DepMap22Q2_geneexp/sample_info.csv', index_col = 0)
cell_line_info

,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,sample_collection_site,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000016,SLR 21,SLR21,SLR21_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_V607,NaN,kidney,...,NaN,NaN,NaN,NaN,NaN,PT-JnARLB,NaN,Clear cell renal cell carcinoma,C4033,NaN
ACH-000032,MHH-CALL-3,MHHCALL3,MHHCALL3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,DSMZ,CVCL_0089,NaN,bone_marrow,...,b_cell,NaN,NaN,NaN,NaN,PT-p2KOyI,NaN,Childhood B acute lymphoblastic leukemia,C9140,NaN
ACH-000033,NCI-H1819,NCIH1819,NCIH1819_LUNG,NaN,NaN,Female,Academic lab,CVCL_1497,NaN,lymph_node,...,NSCLC_adenocarcinoma,NaN,NaN,NaN,NaN,PT-9p1WQv,NaN,Lung adenocarcinoma,C3512,NaN
ACH-000043,Hs 895.T,HS895T,HS895T_FIBROBLAST,NaN,NaN,Female,ATCC,CVCL_0993,NaN,fibroblast,...,NaN,NaN,2D: adherent,NaN,NaN,PT-rTUVZQ,NaN,Melanoma,C3224,NaN
ACH-000049,HEK TE,HEKTE,HEKTE_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_WS59,NaN,kidney,...,NaN,NaN,NaN,immortalized,NaN,PT-qWYYgr,NaN,NaN,NaN,No information is available about this cell li...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002393,CRO-AP3,CROAP3,CROAP3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,Sanger,CVCL_1810,NaN,ascites,...,b_cell_primary_effusion,NaN,NaN,NaN,NaN,PT-TC0lZM,NaN,Primary effusion lymphoma,C6915,NaN
ACH-002394,GEO,GEO,GEO_LARGE_INTESTINE,NaN,NaN,NaN,Sanger,CVCL_0271,NaN,large_intestine,...,NaN,NaN,NaN,NaN,NaN,PT-Fa1q9q,NaN,Colon carcinoma,C4910,NaN
ACH-002395,HuH-6 Clone 5,HUH6CLONE5,HUH6CLONE5_LIVER,NaN,NaN,Male,Sanger,CVCL_1296,NaN,liver,...,NaN,NaN,NaN,NaN,NaN,PT-TtIXsL,ACH-000671,Hepatoblastoma,C3728,NaN


In [38]:
dict_cell_type = dict(zip(cell_line_info.index, cell_line_info["primary_disease"]))
dict_cell_type

{'ACH-000016': 'Kidney Cancer',
 'ACH-000032': 'Leukemia',
 'ACH-000033': 'Lung Cancer',
 'ACH-000043': 'Non-Cancerous',
 'ACH-000049': 'Non-Cancerous',
 'ACH-000051': 'Sarcoma',
 'ACH-000064': 'Non-Cancerous',
 'ACH-000068': 'Lymphoma',
 'ACH-000071': 'Non-Cancerous',
 'ACH-000076': 'Leukemia',
 'ACH-000077': 'Lymphoma',
 'ACH-000079': 'Non-Cancerous',
 'ACH-000084': 'Leukemia',
 'ACH-000089': 'Colon/Colorectal Cancer',
 'ACH-000093': 'Pancreatic Cancer',
 'ACH-000110': 'Gastric Cancer',
 'ACH-000119': 'Non-Cancerous',
 'ACH-000122': 'Leukemia',
 'ACH-000125': 'Non-Cancerous',
 'ACH-000130': 'Leukemia',
 'ACH-000140': 'Lymphoma',
 'ACH-000154': 'Non-Cancerous',
 'ACH-000158': 'Lymphoma',
 'ACH-000165': 'Non-Cancerous',
 'ACH-000166': 'Leukemia',
 'ACH-000170': 'Non-Cancerous',
 'ACH-000172': 'Rhabdoid',
 'ACH-000173': 'Endometrial/Uterine Cancer',
 'ACH-000175': 'Non-Cancerous',
 'ACH-000179': 'Lung Cancer',
 'ACH-000180': 'Non-Cancerous',
 'ACH-000185': 'Non-Cancerous',
 'ACH-000194'

In [39]:
GDSC_selectomics["cancer_type"] = GDSC_selectomics["cell_line"].map(dict_cell_type)
GDSC_selectomics

,cell_line,CELL_LINE_NAME,gdsc_name,IC50,SMILES,cancer_type
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Skin Cancer
2108,ACH-000905,5637,5-AZACYTIDINE,3.783443,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
2109,ACH-000242,RT4,5-AZACYTIDINE,2.648009,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
...,...,...,...,...,...,...
230607,ACH-000989,SNU-175,ZOLEDRONATE,3.225748,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer


In [40]:
GDSC_selectomics.isna().sum()

cell_line         0
CELL_LINE_NAME    0
gdsc_name         0
IC50              0
SMILES            0
cancer_type       0
dtype: int64

## Remove cell line and cancer type with data point less than no.

In [41]:
grouped_df = GDSC_selectomics.groupby("gdsc_name").size().reset_index(name='count')
sorted_df = grouped_df.sort_values(by='count', ascending=False)
print(sorted_df)

          gdsc_name  count
121          MG-132    683
127         MK-2206    682
191   STAUROSPORINE    682
1    5-FLUOROURACIL    682
151       PD0325901    682
..              ...    ...
203       TRETINOIN    129
188          SL0101    129
46            BX795    129
51       CHIR-99021    129
48       CCT-018159    129

[228 rows x 2 columns]


In [42]:
grouped_df = GDSC_selectomics.groupby("cell_line").size().reset_index(name='count')
sorted_df = grouped_df.sort_values(by='count', ascending=False)
print(sorted_df)

      cell_line  count
15   ACH-000030    228
320  ACH-000552    228
387  ACH-000651    228
342  ACH-000580    228
121  ACH-000219    227
..          ...    ...
481  ACH-000804    138
615  ACH-000999    105
442  ACH-000743     88
436  ACH-000725     13
413  ACH-000689     11

[683 rows x 2 columns]


In [43]:
grouped_df = GDSC_selectomics.groupby("cancer_type").size().reset_index(name='count')
sorted_df = grouped_df.sort_values(by='count', ascending=False)
print(sorted_df)

                   cancer_type  count
16                 Lung Cancer  27243
14                    Leukemia  10464
5                Breast Cancer   9046
7      Colon/Colorectal Cancer   8719
17                    Lymphoma   8609
4                 Brain Cancer   7444
26                 Skin Cancer   7111
21              Ovarian Cancer   6656
10           Esophageal Cancer   5901
22           Pancreatic Cancer   5656
11              Gastric Cancer   4609
12        Head and Neck Cancer   4244
13               Kidney Cancer   3405
3                  Bone Cancer   3366
19               Neuroblastoma   3242
2               Bladder Cancer   3240
18                     Myeloma   3123
9   Endometrial/Uterine Cancer   2900
15                Liver Cancer   2754
25                     Sarcoma   2311
6              Cervical Cancer   2299
27              Thyroid Cancer   2290
23             Prostate Cancer   1238
24                    Rhabdoid    637
1             Bile Duct Cancer    213
0           

In [44]:
grouped_df = GDSC_selectomics.groupby("cancer_type")['cell_line'].nunique()
print(grouped_df)

cancer_type
Adrenal Cancer                  1
Bile Duct Cancer                1
Bladder Cancer                 16
Bone Cancer                    16
Brain Cancer                   37
Breast Cancer                  45
Cervical Cancer                11
Colon/Colorectal Cancer        42
Embryonal Cancer                1
Endometrial/Uterine Cancer     14
Esophageal Cancer              29
Gastric Cancer                 23
Head and Neck Cancer           21
Kidney Cancer                  17
Leukemia                       53
Liver Cancer                   13
Lung Cancer                   137
Lymphoma                       43
Myeloma                        15
Neuroblastoma                  17
Non-Cancerous                   1
Ovarian Cancer                 34
Pancreatic Cancer              28
Prostate Cancer                 6
Rhabdoid                        3
Sarcoma                        12
Skin Cancer                    36
Thyroid Cancer                 11
Name: cell_line, dtype: int64


In [45]:
# discard cancer type with cell line count less than 5
cell_line_count = GDSC_selectomics.groupby("cancer_type")['cell_line'].nunique().reset_index(name='count')
cell_line_count = cell_line_count[cell_line_count['count'] >= 5]
cell_lines = cell_line_count['cancer_type'].tolist()
GDSC_selectomics = GDSC_selectomics[GDSC_selectomics['cancer_type'].isin(cell_lines)]
GDSC_selectomics

,cell_line,CELL_LINE_NAME,gdsc_name,IC50,SMILES,cancer_type
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Skin Cancer
2108,ACH-000905,5637,5-AZACYTIDINE,3.783443,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
2109,ACH-000242,RT4,5-AZACYTIDINE,2.648009,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
...,...,...,...,...,...,...
230607,ACH-000989,SNU-175,ZOLEDRONATE,3.225748,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer


In [46]:
grouped_df = GDSC_selectomics.groupby("cancer_type")['cell_line'].nunique()
print(grouped_df)

cancer_type
Bladder Cancer                 16
Bone Cancer                    16
Brain Cancer                   37
Breast Cancer                  45
Cervical Cancer                11
Colon/Colorectal Cancer        42
Endometrial/Uterine Cancer     14
Esophageal Cancer              29
Gastric Cancer                 23
Head and Neck Cancer           21
Kidney Cancer                  17
Leukemia                       53
Liver Cancer                   13
Lung Cancer                   137
Lymphoma                       43
Myeloma                        15
Neuroblastoma                  17
Ovarian Cancer                 34
Pancreatic Cancer              28
Prostate Cancer                 6
Sarcoma                        12
Skin Cancer                    36
Thyroid Cancer                 11
Name: cell_line, dtype: int64


In [47]:
# discard cell line with data point less than 10
cell_line_count = GDSC_selectomics.groupby("cell_line").size().reset_index(name='count')
cell_line_count = cell_line_count[cell_line_count['count'] >= 10]
cell_lines = cell_line_count['cell_line'].tolist()
GDSC_selectomics = GDSC_selectomics[GDSC_selectomics['cell_line'].isin(cell_lines)]
GDSC_selectomics

,cell_line,CELL_LINE_NAME,gdsc_name,IC50,SMILES,cancer_type
2103,ACH-000052,A673,5-AZACYTIDINE,2.165547,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2106,ACH-000087,SK-ES-1,5-AZACYTIDINE,2.933776,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bone Cancer
2107,ACH-000644,COLO-829,5-AZACYTIDINE,4.435078,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Skin Cancer
2108,ACH-000905,5637,5-AZACYTIDINE,3.783443,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
2109,ACH-000242,RT4,5-AZACYTIDINE,2.648009,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
...,...,...,...,...,...,...
230607,ACH-000989,SNU-175,ZOLEDRONATE,3.225748,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230608,ACH-000955,SNU-407,ZOLEDRONATE,3.834554,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230609,ACH-000532,SNU-61,ZOLEDRONATE,4.399959,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer
230610,ACH-000991,SNU-81,ZOLEDRONATE,3.612773,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Colon/Colorectal Cancer


In [48]:
grouped_df = GDSC_selectomics.groupby("cell_line").size().reset_index(name='count')
sorted_df = grouped_df.sort_values(by='count', ascending=False)
print(sorted_df)
grouped_df = GDSC_selectomics.groupby("gdsc_name").size().reset_index(name='count')
sorted_df = grouped_df.sort_values(by='count', ascending=False)
print(sorted_df)

      cell_line  count
383  ACH-000651    228
317  ACH-000552    228
15   ACH-000030    228
339  ACH-000580    228
118  ACH-000219    227
..          ...    ...
340  ACH-000581    138
610  ACH-000999    105
438  ACH-000743     88
432  ACH-000725     13
409  ACH-000689     11

[676 rows x 2 columns]
          gdsc_name  count
121          MG-132    676
127         MK-2206    675
191   STAUROSPORINE    675
1    5-FLUOROURACIL    675
151       PD0325901    675
..              ...    ...
203       TRETINOIN    128
188          SL0101    128
46            BX795    128
51       CHIR-99021    128
48       CCT-018159    128

[228 rows x 2 columns]


In [49]:
GeneExp = GeneExp.loc[list(MultiOmics_CellLine)]
processed_drug_df = processed_drug_df.loc[processed_drug_df["DRUG_NAME"].isin(GDSC_selectomics["gdsc_name"])]
processed_drug_df

,DRUG_ID,DRUG_NAME,SMILES
0,1003,CAMPTOTHECIN,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
968,1004,VINBLASTINE,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...
1709,1005,CISPLATIN,N.N.Cl[Pt]Cl
2469,1006,CYTARABINE,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O
3212,1007,DOCETAXEL,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...
...,...,...,...
235232,2177,SGC0946,CC(C)N(CCCNC(=O)NC1=CC=C(C=C1)C(C)(C)C)CC2C(C(...
236178,2359,GSK2830371,CC1=C(C=C(C=N1)Cl)NCC2=CC=C(S2)C(=O)NC(CC3CCCC...
239834,2439,GLUTATHIONE,C(CC(=O)NC(CS)C(=O)NCC(=O)O)C(C(=O)O)N
240566,2498,ALPHA-LIPOIC ACID,C1CSSC1CCCCC(=O)O


In [50]:
print(processed_drug_df.columns)
print(type(processed_drug_df.columns))
print([type(c) for c in processed_drug_df.columns])

Index(['DRUG_ID', 'DRUG_NAME', 'SMILES'], dtype='object')
<class 'pandas.core.indexes.base.Index'>
[<class 'str'>, <class 'str'>, <class 'str'>]


In [51]:
processed_drug_df.to_csv("datasets/drug_info.csv", index=False)

In [52]:
print(type(processed_drug_df.columns))
print(list(processed_drug_df.columns))

<class 'pandas.core.indexes.base.Index'>
['DRUG_ID', 'DRUG_NAME', 'SMILES']


## Check data before export

In [53]:
# Drugsens_check
DrugSens_GDSC = GDSC_selectomics.reset_index(drop=True)
print(DrugSens_GDSC.duplicated(subset=["cell_line", "gdsc_name"]).sum())
print(DrugSens_GDSC.isna().sum().sum())
print(DrugSens_GDSC.nunique())

0
0
cell_line            676
CELL_LINE_NAME       676
gdsc_name            228
IC50              134103
SMILES               227
cancer_type           23
dtype: int64


In [54]:
# Omics_check
print(GeneExp.isna().sum().sum(), GeneExp.shape)

0 (683, 735)


In [55]:
# find %missing
DrugSens_pivotted = DrugSens_GDSC.pivot(index = 'gdsc_name',columns = 'cell_line',values='IC50')
missingvaluespercent = DrugSens_pivotted.isna().sum().sum()*100/(len(DrugSens_pivotted.index)*len(DrugSens_pivotted.columns))
print(f'Across {len(DrugSens_pivotted.columns)} cell lines and {len(DrugSens_pivotted.index)} drugs, DrugSens dataset from GDSC, with a total combination of {DrugSens_pivotted.notna().sum().sum()},has missing values of {round(missingvaluespercent,2)} percent.')

Across 676 cell lines and 228 drugs, DrugSens dataset from GDSC, with a total combination of 135870,has missing values of 11.85 percent.


In [56]:
DrugSens_pivotted.index.name = ''
DrugSens_pivotted.columns.name = 'gdsc_name'

In [57]:
DrugSens_pivotted

gdsc_name,ACH-000001,ACH-000002,ACH-000004,ACH-000006,ACH-000007,ACH-000008,ACH-000009,ACH-000012,ACH-000015,ACH-000018,...,ACH-001653,ACH-001654,ACH-001655,ACH-001674,ACH-001702,ACH-001703,ACH-001709,ACH-001711,ACH-001715,ACH-001716
,,,,,,,,,,,,,,,,,,,,,
5-AZACYTIDINE,2.546236,0.918406,NaN,-0.025091,3.381085,1.200924,4.549504,3.596017,1.767661,NaN,...,4.444854,4.985248,2.438524,2.442553,2.809720,1.072637,0.736222,NaN,2.119401,3.139363
5-FLUOROURACIL,4.831189,2.558821,2.460608,0.073319,1.741203,3.944557,5.537326,5.188120,4.859701,3.666305,...,4.667966,6.834709,5.647135,5.206538,4.026975,2.264862,1.507962,5.569870,4.702442,2.482624
A-366,5.116512,4.825807,NaN,2.943904,3.638446,5.463790,4.873022,6.017634,5.019106,NaN,...,5.532511,5.805159,5.388174,4.199525,5.612776,3.555867,3.061064,NaN,3.922953,3.976601
ABT737,2.801066,-2.814475,0.728768,-0.381245,NaN,0.477799,1.673147,4.386754,2.205683,2.924887,...,0.269580,4.686926,3.633314,-2.732603,5.396935,-1.950937,-0.089296,0.008803,3.071056,3.219442
ACETALAX,0.963399,5.546654,4.986483,NaN,5.765172,7.003308,4.190445,7.574307,2.667541,NaN,...,6.361515,8.035360,7.171101,6.057721,6.464023,5.894741,4.689977,5.074948,5.553689,6.372520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WZ4003,5.261262,3.664652,3.186098,NaN,3.225217,2.080867,4.314370,3.105553,3.361504,5.697644,...,4.909269,5.769843,3.905051,2.153281,5.376596,2.842662,2.584390,2.222808,1.793008,2.643982
XAV939,5.179876,4.344360,3.142230,NaN,4.564638,4.341081,6.499375,3.176033,4.029829,4.217360,...,4.115210,5.972627,3.853159,4.607319,4.617363,3.079182,3.063853,3.301476,3.430986,4.607874
YK-4-279,1.549981,0.318175,2.024362,0.336980,NaN,1.881527,4.162697,4.200521,1.412631,2.332873,...,0.488395,3.832124,3.578193,0.918643,2.929148,0.229974,0.430738,1.867553,2.552756,1.518968


## Data from Pivot with null

In [58]:
DrugSens_from_pivotted_withnull = DrugSens_pivotted.stack(dropna=False)
DrugSens_from_pivotted_withnull = DrugSens_from_pivotted_withnull.reset_index(name='IC50')
DrugSens_from_pivotted_withnull = DrugSens_from_pivotted_withnull.reset_index(drop=True)
DrugSens_from_pivotted_withnull = DrugSens_from_pivotted_withnull.rename(columns={"": "gdsc_name", "gdsc_name": "cell_line"})
DrugSens_from_pivotted_withnull = DrugSens_from_pivotted_withnull[['cell_line', 'gdsc_name', 'IC50']]

dict_SMILES = dict(zip(DrugSens_GDSC["gdsc_name"], DrugSens_GDSC["SMILES"]))
dict_cell_type = dict(zip(cell_line_info.index, cell_line_info["primary_disease"]))
DrugSens_from_pivotted_withnull["SMILES"] = DrugSens_from_pivotted_withnull["gdsc_name"].map(dict_SMILES)
DrugSens_from_pivotted_withnull["cancer_type"] = DrugSens_from_pivotted_withnull["cell_line"].map(dict_cell_type)
DrugSens_from_pivotted_withnull

/var/folders/dx/n9ptww816r99pv93j_qb32hr0000gn/T/ipykernel_25815/3303588704.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  DrugSens_from_pivotted_withnull = DrugSens_pivotted.stack(dropna=False)


,cell_line,gdsc_name,IC50,SMILES,cancer_type
0,ACH-000001,5-AZACYTIDINE,2.546236,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Ovarian Cancer
1,ACH-000002,5-AZACYTIDINE,0.918406,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Leukemia
2,ACH-000004,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Leukemia
3,ACH-000006,5-AZACYTIDINE,-0.025091,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Leukemia
4,ACH-000007,5-AZACYTIDINE,3.381085,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Colon/Colorectal Cancer
...,...,...,...,...,...
154123,ACH-001703,ZOLEDRONATE,3.487607,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Lymphoma
154124,ACH-001709,ZOLEDRONATE,3.060058,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Lymphoma
154125,ACH-001711,ZOLEDRONATE,1.453652,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Brain Cancer
154126,ACH-001715,ZOLEDRONATE,3.310160,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Bone Cancer


In [59]:
DrugSens_from_pivotted_onlynull = DrugSens_from_pivotted_withnull[DrugSens_from_pivotted_withnull['IC50'].isna()]
DrugSens_from_pivotted_onlynull.reset_index(drop=True, inplace=True)
DrugSens_from_pivotted_onlynull

,cell_line,gdsc_name,IC50,SMILES,cancer_type
0,ACH-000004,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Leukemia
1,ACH-000018,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Bladder Cancer
2,ACH-000053,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Lymphoma
3,ACH-000056,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Lymphoma
4,ACH-000058,5-AZACYTIDINE,NaN,Nc1ncn(C2OC(CO)C(O)C2O)c(=O)n1,Thyroid Cancer
...,...,...,...,...,...
18253,ACH-001530,ZOLEDRONATE,NaN,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Endometrial/Uterine Cancer
18254,ACH-001542,ZOLEDRONATE,NaN,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Head and Neck Cancer
18255,ACH-001605,ZOLEDRONATE,NaN,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Brain Cancer
18256,ACH-001617,ZOLEDRONATE,NaN,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O,Lymphoma


## Export data

In [60]:
os.makedirs("datasets/features/", exist_ok = True)
os.makedirs("datasets/sensitivity/", exist_ok = True)

GeneExp.to_csv('datasets/features/GeneExp.csv')
DrugSens_GDSC.to_csv('datasets/sensitivity/DrugSens.csv', index=True)
DrugSens_pivotted.to_csv('datasets/sensitivity/DrugSensPivotted.csv')
DrugSens_from_pivotted_withnull.to_csv('datasets/sensitivity/DrugSens_withnull.csv', index=True)
DrugSens_from_pivotted_onlynull.to_csv('datasets/sensitivity/DrugSens_onlynull.csv', index=True)

In [61]:
with open('datasets/drug_name.txt','w') as drug_name_flie:
   drug_name_flie.write('\n'.join(DrugSens_pivotted.index.tolist()))